In [1]:
import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [2]:
import os

cache_dir = "/root/autodl-fs"
os.environ["TRANSFORMERS_CACHE"] = cache_dir

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

/root/miniconda3/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
import re, string

In [5]:
model_path = "JiazhenLiu01/falcon-7b-threeturn"
tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map={"": 0},
    torch_dtype='auto',
    #offload_folder="offload"
    cache_dir=cache_dir
).eval()

/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/522M [00:00<?, ?B/s]

In [6]:
# Read questions from question.txt file and extract them
questions = []
with open("evaluation/evaluation.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        # Extract questions after the question number
        question = line.strip()
        questions.append(question)

In [7]:
for question in questions:
    print(question+"\n")

Could you please share your full name and age with me?

Can you tell me a bit about your family background? Do you have any other family members at home?

What do you typically enjoy doing in your daily life? Do you have any particular interests or hobbies?

I'd like to learn about your school life. Which school do you attend, and how do you feel about it?

Do you have any specific career or vocational goals? What kind of work would you like to do in the future?

Could you tell me about your friends or social circle? Do you have any close friends?

Have you experienced any past mental health issues or therapy? Do you feel these experiences have any impact on your current state?

How would you describe your emotional and mood states in your daily life? Do you sometimes feel low or anxious?

Let's talk about your family relationships. How would you describe your relationships with your family members? Do you feel family dynamics affect your mood?

Lastly, Tom, what do you hope to gain fr

In [8]:
def extract_dialogue(text):
    tom_index = text.find("Tom:")
    if tom_index == -1:
        return None

    psychologist_index = min(text.find('Psychologist:',tom_index),text.find('Psychologists:',tom_index),text.find('Psychologist',tom_index),text.find('Psychologists',tom_index),
                             text.find('Psychiatrist:',tom_index),text.find('Psychiatrists:',tom_index),text.find('Psychiatrist',tom_index),text.find('Psychiatrists',tom_index),
                             text.find('psychologist:',tom_index),text.find('psychologists:',tom_index),text.find('psychologist',tom_index),text.find('psychologists',tom_index),
                             text.find('psychiatrist:',tom_index),text.find('psychiatrists:',tom_index),text.find('psychiatrist',tom_index),text.find('psychiatrists',tom_index),
                             text.find('\\n',tom_index),text.find('\n',tom_index))
                                       
    if psychologist_index != -1:
        dialogue = text[tom_index + len("Tom:"):psychologist_index]
    else:
        dialogue = text[tom_index + len("Tom:"):]

    last_period_index = max(dialogue.rfind('.'),dialogue.rfind('--'),dialogue.rfind('?'))

    dialogue = dialogue[:last_period_index+1]
    
    return dialogue.strip()

In [9]:
# Loop for dialogue inference
with open("evaluation/falcon-threeturn.txt", "a") as output_file:
    for question in questions:
        input_text = f"<<SYS>>Pentending Your name is Tom, and you are now communicating with a psychologist in a therapy session. Respond to the psychologist as Tom using English words and respond in a casual tone. Your reply should be no longer than three sentences.<</SYS>>\n[INST] Psychologist:{question} [/INST]\nTom: "
        # Encode input text into tokens
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        input_ids = input_ids.to("cuda")
        # Use the trained model for dialogue inference
        output = model.generate(input_ids, repetition_penalty=3.0, pad_token_id=tokenizer.eos_token_id,max_new_tokens=80)

        # Decode the model output tokens into text
        output_text1 = tokenizer.decode(output[0], skip_special_tokens=True)

        output_text = output_text1.splitlines()[2]
        
        response = extract_dialogue(output_text)

        # print("Original response:", output_text1)
        print("Generated response:", response)

        # Append each answer to falcon_finetuned.txt
        output_file.write("\n"+ response)

Generated response: 16 years old; my names-- this will make it easier if I start from here because sometimes when we go by year level they get confused between us two tomes out there's an other one that goes into detail about what subjects he has for his schooling or whatever..Therapy can help young people manage their problems better so they don't have trouble later on in life.
Generated response: 00 minutes left on this one.. There's Dad-- obviously Mom because we only just got off her--"The interview continued for another hour while I took notes of what was said by both him and his mother during their first visit together since he had been out there two weeks ago.
Generated response: 00 minutes of exercise five days-- well,. I love playing football; that's what I normally would watch on TV every Sunday morning at home when we used to wake up early enough for it. So yeah, exercising most weekends because dad does run around like crazy sometimes during Saturdays and Sundays if there’s